## .NET多线程

### 基本概念

<font color="orange">什么是线程</font>
- 线程时操作系统中能够独立运行的最小单位，也是程序中能够并发执行的一段指令序列
- 线程也是进程的一部分，一个进程可以包含多个线程，这些线程共享进程的资源
- 进程有入口线程，也可以创建更多的线程

<font color="orange">为什么要多线程</font>
- 批量重复任务希望同时进行（比如对于数组中的每个元素都进行相同且耗时的操作）
- 多个不同任务希望同时进行，互不干扰（比如有多个后台线程需要做轮询等操作）

<font color="orange">什么是多线程</font>
- 一组预先创建的线程，可以重复使用来执行多个任务
- 避免频繁地创建和销毁线程，从而减少了线程创建和销毁的开销
- 异步编程默认使用线程池

<font color="orange">什么是线程安全</font>
- <font color="orange">线程安全</font>
    - 多个线程访问共享资源时，对共享资源的访问不会导致数据不一致或不可预期的结果
- <font color="orange">同步机制</font>
    - 用于协调和控制多个线程之间执行顺序和互斥访问共享资源
    - 确保线程按照特定的顺序执行，避免竞态条件和数据不一致的问题
- <font color="orange">原子操作</font>
    - 在执行过程中不会被中断的操作，不可分割，要么完全执行，要么完全不执行，没有中间状态
    - 在多线程环境下，原子操作能够保证数据的一致性和可靠性，避免出现竞态条件和数据竞争的问题

In [7]:
using System.Threading;

object lockObj = new object();  //加锁之后可以避免数据不一致的问题
const int total = 1000_000;

int count = 0;

var thread1 = new Thread(ThreadMethod);
var thread2 = new Thread(ThreadMethod);

thread1.Start();
thread2.Start();

thread1.Join();
thread2.Join();
//Count:1356790
Console.WriteLine($"Count:{count}");

void ThreadMethod()
{
    for(int i = 0; i < total; i++) 
        Interlocked.Increment(ref count); //原子操作 C#底层提供的方法
        //lock(lockObj) count++; //非原子操作
}

Count:2000000


In [ ]:
using System.Threading;

var queue = new Queue<int>();
object lockObj = new object();
var producer = new Thread(AddNumbers);
var consumer1 = new Thread(ReadNumbers);
var consumer2 = new Thread(ReadNumbers);

producer.Start();
consumer1.Start();
consumer2.Start();

producer.Join();
consumer1.Interrupt();
consumer2.Interrupt();
consumer1.Join();
consumer2.Join();

void AddNumbers()
{
    for(int i = 0; i < 20; i++)
    {
        Thread.Sleep(20);
        queue.Enqueue(i);
    }
}

void ReadNumbers()
{
    try
    {
        while(true)
        {
            lock(lockObj)
            {
                if(queue.TryDequeue(out var res))
                    Console.WriteLine(res);
            }
            Thread.Sleep(1);

        }
    }
    catch(ThreadInterruptedException)
    {
        Console.WriteLine("Thread Interrupted.");
    }
}

<font color="orange">常用实现方式</font>
- 线程
- 线程池
- 异步编程
- 自带方法
    - Parallel
        - For
        - ForEach
        - Invoke
    - PLINQ
        - AsParallel
        - AsSequential
        - AsOrdered

In [13]:
using System.Diagnostics;

var inputs = Enumerable.Range(1,20).ToArray();
var outputs = new int[inputs.Length];

var sw = Stopwatch.StartNew();

//串行方式
// for(int i = 0; i < inputs.Length; i++)
// {
//     outputs[i] = HeavyJob(inputs[i]);
// }

//并行方式
//Parallel.For(0,inputs.Length, i => outputs[i] = HeavyJob(inputs[i]));

//PLINQ
outputs = inputs.AsParallel().Select(x => HeavyJob(x)).ToArray();

Console.WriteLine($"Elapsed time: {sw.ElapsedMilliseconds}ms");

PrintArray(outputs);

int HeavyJob(int input)
{
    Thread.Sleep(100);
    return input * input;
}

void PrintArray<T>(T[] arr){
    foreach(var item in arr)
        Console.Write($"{item}, ");
}

Elapsed time: 223ms
1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 

### 线程的使用方法

<font color="cyan">线程的创建</font>
- 创建Thread实例，并传入ThreadStart委托————还可以配置线程，如是否为后台线程
- 调用ThreadStart方法，支持传参

In [14]:
var thread1 = new Thread(ThreadMethod1);
thread1.Start();

var thread2 = new Thread(ThreadMethod2);
thread2.Start(123); //可以在这里传参

void ThreadMethod1(){}
void ThreadMethod2(object? obj){}


<font color="cyan">线程的终止</font>
- 调用Thread.Join 方法，还可以传参
- 调用Thread.Interupt 方法，中断线程的执行
    - 会在线程中抛出ThreadInterruptedException
    - 如果线程中包含一个while(true)循环，那么需要保证包含等待方法，如IO操作，Thread.Sleep等
- 不能用Abort? 
    - 会立刻强制终止线程，可能导致一些严重的问题，包括资源泄露和不可预测的行为
    - 较新版本的.NET中如果使用这个方法，会报PlatformNotSupportedException
    - 推荐使用Thread.Interrupt或CancellationToken

In [16]:
var th = new Thread(
    () => {
        for(int i = 0; i < 20; i++)
        {
            Thread.Sleep(200);
            Console.WriteLine("Thread is still running...");
        }
    }
){IsBackground = true, Priority = ThreadPriority.Normal};

th.Start();
Console.WriteLine("In main thread, waiting for thread to finish...");
th.Join();//子线程会在此阻塞主线程
Console.WriteLine("Done...");

In main thread, waiting for thread to finish...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Done...


In [18]:
var th = new Thread(
    () => {
        try
        {
            for(int i = 0; i < 20; i++)
            {
                Thread.Sleep(200);
                Console.WriteLine("Thread is still running...");
            }
        }
        catch(ThreadInterruptedException)
        {
            Console.WriteLine("Thread is Interrupted...");
        }
        finally{

        }
}){IsBackground = true, Priority = ThreadPriority.Normal};

th.Start();
Console.WriteLine("In main thread, waiting for thread to finish...");
Thread.Sleep(1000);//让线程执行一秒后结束它
th.Interrupt();
th.Join();//子线程会在此阻塞主线程
Console.WriteLine("Done...");

In main thread, waiting for thread to finish...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is still running...
Thread is Interrupted...
Done...


<font color="cyan">线程的挂起与恢复</font>
- Thread.Suspend以及Thread.Resume
- 较新版本的.NET中，这两个方法已经被标记为Obsolete,且调用会报错
- 推荐使用锁、信号量等方式实现这一逻辑

### 线程安全与同步机制

<font color="green">原子操作</font>

<font color="green">锁与信号量</font>
- lock & Monitor
- Mutex  互斥锁 可以进程间共享
- Semaphore  用于线程间的同步,可以进程间共享
- WaitHandle
    - ManualResetEvent 
    - AutoResetEvent 
- ReaderWriterLock 读写锁 允许多个Reader读，只允许一个Writer写，Writer与Reader互斥


In [20]:
using System.Diagnostics;

var inputs = Enumerable.Range(1,20).ToArray();
var outputs = new int[inputs.Length];
//开始有三个窗口空闲，最多有3个窗口可以通过，窗口都被占满后后续的线程都将被阻塞
var semaphore = new Semaphore(3,3);

var sw = Stopwatch.StartNew();

//串行方式
// for(int i = 0; i < inputs.Length; i++)
// {
//     outputs[i] = HeavyJob(inputs[i]);
// }

//并行方式
//Parallel.For(0,inputs.Length, i => outputs[i] = HeavyJob(inputs[i]));

//PLINQ
outputs = inputs.AsParallel().Select(x => HeavyJob(x)).ToArray();

Console.WriteLine($"Elapsed time: {sw.ElapsedMilliseconds}ms");
semaphore.Dispose();//用完后要释放
PrintArray(outputs);

int HeavyJob(int input)
{
    semaphore.WaitOne();//等待一个空闲窗口
    Thread.Sleep(100);
    semaphore.Release();//窗口用完后要释放
    return input * input;
}

void PrintArray<T>(T[] arr){
    foreach(var item in arr)
        Console.Write($"{item}, ");
}

Elapsed time: 755ms
1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361, 400, 

<font color="green">轻量型</font>
- SemaphoreSlim
- ManualResetEventSlim
- ReaderWriterLockSlim

<font color="green">不要自己造轮子</font>
- 线程安全的单例：Lazy
- 线程安全的集合类型：ConcurrentBag、ConcurrentStack、ConcurrentQueue、ConcurrentDictionary
- 阻塞集合：BlockingCollection
- 通道: Channel
- 原子操作：Interlocked
- 周期任务：PeriodicTimer

In [22]:
using System.Threading;
using System.Collections.Concurrent;
//线程安全
var queue = new ConcurrentQueue<int>();
object lockObj = new object();
var producer = new Thread(AddNumbers);
var consumer1 = new Thread(ReadNumbers);
var consumer2 = new Thread(ReadNumbers);

producer.Start();
consumer1.Start();
consumer2.Start();

producer.Join();
consumer1.Interrupt();
consumer2.Interrupt();
consumer1.Join();
consumer2.Join();

void AddNumbers()
{
    for(int i = 0; i < 20; i++)
    {
        Thread.Sleep(20);
        queue.Enqueue(i);
    }
}

void ReadNumbers()
{
    try
    {
        while(true)
        {
            
            if(queue.TryDequeue(out var res))
                Console.WriteLine(res);
            Thread.Sleep(1);

        }
    }
    catch(ThreadInterruptedException)
    {
        Console.WriteLine("Thread Interrupted.");
    }
}

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Thread Interrupted.
Thread Interrupted.
